In [203]:
# !pip3 install --upgrade pip
# !pip3 install --upgrade pandas
# !pip3 install --upgrade sklearn
# !pip3 install --upgrade pymysql
# !pip3 install --upgrade sqlalchemy
# !pip3 install --upgrade pyodb

In [204]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None 
from keras.models import load_model
import requests
import json
import datetime
import pytz
import pymysql
import warnings
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

In [205]:
conn=pymysql.connect(
    host='sql10.freemysqlhosting.net',
    port=int(3306),
    user='sql10489794',
    passwd='IPELfw5A2X',
    db='sql10489794'
)

In [206]:
time_now = datetime.datetime.now(tz=pytz.timezone("Europe/Berlin"))
key = "JPRR7UQ66H89YQ23GM3TYFWBU"
date_start = (time_now - datetime.timedelta(days=2)).strftime("%Y-%m-%d") 
date_stop  = (time_now).strftime("%Y-%m-%d")
result = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/krak%C3%B3w/{date_start}/{date_stop}?unitGroup=metric&include=hours&key={key}&contentType=json")
# print(str(result.content.decode('utf-8')))
json_dict = json.loads(str(result.content.decode('utf-8')))
new_dict = {}
i = 0


for day in json_dict['days'][::-1]:
    for hour in day['hours'][::-1]:
        if day['datetime'] != time_now.strftime("%Y-%m-%d") or hour['datetime'] < time_now.strftime("%H:%M:00"):
            new_dict[i] = hour.copy()
            new_dict[i]['datetime'] = day['datetime'] + 'T' + backup_hour
#             print(day['datetime'], hour['datetime'],"---", new_dict[i]['datetime'])
            query = f"SELECT * FROM esp WHERE weather_time = '{day['datetime']}' AND hour = '{hour['datetime'].split(':')[0]}'"
#             print(query)
            esp_measurement=pd.read_sql_query(query, conn)
            if not esp_measurement.empty:
                print(esp_measurement)
                new_dict[i]['temp'] = esp_measurement['temperature'][0]
                new_dict[i]['humidity'] = esp_measurement['humidity'][0]
            i=i+1
            
measured_data_wlabels = pd.DataFrame.from_dict(new_dict).transpose()[:48]

/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   id  temperature  humidity  hour weather_time
0  21           21        41    15   2022-05-05


/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   id  temperature  humidity  hour weather_time
0  20           24        31    14   2022-05-05


/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


...

/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   id  temperature  humidity  hour weather_time
0   3           16        39    21   2022-05-04


/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 conne

   id  temperature  humidity  hour weather_time
0   1           21        41    22   2022-05-03


/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 conne

In [207]:
measured_data_wlabels

,datetime,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,...,visibility,cloudcover,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,source
0,2022-05-05T12:00:00,1651759200,17.8,17.8,73.01,12.9,0.7,85.7,0.0,0.0,...,12.3,100.0,199.0,0.7,2.0,10.0,"Rain, Overcast",rain,None,fcst
1,2022-05-05T12:00:00,1651755600,21,22.0,41,7.0,0.6,100.0,0.0,0.0,...,10.0,25.0,1184.0,4.3,10.0,10.0,"Rain, Partially cloudy",rain,"[EPKT, EPKK, E6527]",obs
2,2022-05-05T12:00:00,1651752000,24,21.0,31,8.9,0.1,100.0,0.0,0.0,...,10.0,25.0,1046.0,3.8,10.0,10.0,"Rain, Partially cloudy",rain,"[EPKT, EPKK, E6527]",obs
3,2022-05-05T12:00:00,1651748400,24,20.9,29,7.9,0.1,100.0,0.0,0.0,...,10.0,25.0,848.0,3.1,8.0,10.0,"Rain, Partially cloudy",rain,"[EPKT, EPKK]",obs
4,2022-05-05T12:00:00,1651744800,27,20.0,24,8.9,0.0,0.0,0.0,0.0,...,10.0,25.0,818.0,2.9,8.0,10.0,Partially cloudy,partly-cloudy-day,"[EPKT, EPKK]",obs
5,2022-05-05T12:00:00,1651741200,40,18.9,13,6.9,0.0,0.0,0.0,0.0,...,10.0,25.0,742.0,2.7,7.0,10.0,Partially cloudy,partly-cloudy-day,"[EPKT, EPKK]",obs
6,2022-05-05T12:00:00,1651737600,40,17.0,13,7.0,0.0,0.0,0.0,0.0,...,10.0,10.6,624.0,2.2,6.0,10.0,Clear,clear-day,"[EPKT, EPKK]",obs
7,2022-05-05T12:00:00,1651734000,27,14.0,23,7.0,0.0,0.0,0.0,0.0,...,10.0,38.3,1168.0,4.2,10.0,10.0,Partially cloudy,partly-cloudy-day,"[EPKT, EPKK, E6527]",obs
8,2022-05-05T12:00:00,1651730400,22,11.0,36,5.0,0.0,0.0,0.0,0.0,...,10.0,67.0,389.0,1.4,4.0,10.0,Partially cloudy,partly-cloudy-day,"[EPKT, EPKK, E6527]",obs
9,2022-05-05T12:00:00,1651726800,15,9.1,53,5.0,0.0,0.0,0.0,0.0,...,10.0,54.6,277.0,1.0,3.0,10.0,Partially cloudy,partly-cloudy-day,"[EPKT, EPKK, E6527]",obs


In [208]:
columns_names = [col for col in measured_data_wlabels]

pd.DataFrame(columns_names, columns=['column_name'])

,column_name
0,datetime
1,datetimeEpoch
2,temp
3,feelslike
4,humidity
5,dew
6,precip
7,precipprob
8,snow
9,snowdepth


In [209]:
import joblib
sc = joblib.load("scaler.save")

In [210]:
selected_columns = [2, 4, 21]
selected_columns_names = [columns_names[col] for col in selected_columns]
print(selected_columns_names)

# dataset = dataset.dropna(subset=selected_columns_names)
# dataset=dataset.reset_index(drop=True)
measured_data_wlabels = measured_data_wlabels.iloc[:,selected_columns]


cond = ['Snow', 'Rain, Partially cloudy', 'Snow, Partially cloudy', 'Rain', 'Partially cloudy', 'Rain, Overcast', 'Snow, Rain, Overcast', 'Overcast', 'Snow, Overcast', 'Clear', 'Snow, Rain, Partially cloudy']
for label in cond:
    measured_data_wlabels[label] = 0
    measured_data_wlabels.loc[measured_data_wlabels['conditions']==label, label] = 1
measured_data_wlabels = measured_data_wlabels.drop(columns='conditions')

measured_data = measured_data_wlabels.values

measured_data = sc.transform(measured_data)
measured_data = np.array(measured_data)
# print(measured_data)
measured_data = np.reshape(measured_data,(1,measured_data.shape[1],measured_data.shape[0]))
# print(measured_data)

['temp', 'humidity', 'conditions']


In [211]:
!tar xzf weather_forecast_model.tgz

In [212]:
model = load_model('weather_forecast_model')

In [213]:
predicted_weather = model.predict(measured_data)
# print(predicted_weather)
# print(predicted_weather.shape)
predicted_weather = np.reshape(predicted_weather, (predicted_weather.shape[2],predicted_weather.shape[1]))
# print(predicted_weather.shape)
predicted_weather = sc.inverse_transform(predicted_weather)

In [214]:
predicted_weather_wlabels = pd.DataFrame(predicted_weather, columns=[col for col in selected_columns_names if col != 'conditions']+cond)
max_cond = np.argmax(predicted_weather[:, 2:], axis=1)
for i, mcond in enumerate(max_cond):
    predicted_weather_wlabels.loc[i, 'conditions'] = cond[mcond]
predicted_weather_wlabels = predicted_weather_wlabels.drop(columns=cond)


In [215]:
predicted_weather_wlabels
predicted_weather_wlabels['datetime'] = [(time_now + datetime.timedelta(hours=i+1)).strftime("%Y-%m-%dT%H:00:00") for i in range(12)]
column_to_move = predicted_weather_wlabels.pop("datetime")
predicted_weather_wlabels.insert(0, "datetime", column_to_move)
predicted_weather_wlabels

,datetime,temp,humidity,conditions
0,2022-05-05T17:00:00,16.187181,54.179478,Partially cloudy
1,2022-05-05T18:00:00,17.407625,48.750874,Partially cloudy
2,2022-05-05T19:00:00,15.587179,51.845032,Partially cloudy
3,2022-05-05T20:00:00,13.810544,52.235435,Partially cloudy
4,2022-05-05T21:00:00,12.780716,52.806553,Partially cloudy
5,2022-05-05T22:00:00,13.408257,61.631062,Partially cloudy
6,2022-05-05T23:00:00,13.397468,66.856613,Partially cloudy
7,2022-05-06T00:00:00,13.629154,72.760086,Clear
8,2022-05-06T01:00:00,9.388839,76.705399,Clear
9,2022-05-06T02:00:00,8.738778,80.138603,Clear


In [216]:
cursor = conn.cursor()
for row in predicted_weather_wlabels[::-1].iterrows():
    date, hour = row[1]['datetime'].split("T")[0], row[1]['datetime'].split("T")[1]
    temp = row[1]['temp']
    humidity = row[1]['humidity'] 
    conditions = row[1]['conditions'] 
    query = f"INSERT INTO forecast (date, hour, temp, humidity, conditions) VALUES ('{date}', '{hour}', {int(temp)}, {int(humidity)}, '{conditions}')"
    cursor.execute(query)
    conn.commit()